In [2]:
# boilerplate
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from pretty import mat_print, mathjax,math
from IPython.display import display, HTML, Latex



import numpy as np
import plotly.graph_objects as go

import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D

display(HTML(mathjax))


# Meta-Análise: Uso da interpolação polinomial para determinar características de performance de um motor à diesel

Alunos: Fábio Melo e Thuane França


### Objeto de Estudo: Motor ANDORIA 4CTi90-1BE6

<img src='ANDORIA-engine.png'>

### Equipamentos utilizados para teste:
* Dinamômetro de Correntes de Foucault. (AVL Dynoperform 160)
* Medidor de massa de combutível (AUTOMEX AMX 212F)
* Equipamentos de Controle e medição ISO 15550:2009

### Medições Feitas:
* X    = Velocidades Distintas de motor (em RPM):1400, 1800, 2200, 2600, 3000, 3400
* F(x) = Poder Efetivo(Pe)/Torque(T)/Consumo de Combustível(B)/Consumo Específico de Combustível(b)

### Resultado das Medições:
<img src='res1.png'>

In [8]:
# plotando o torque
# Aplicação da Interpolação polinomial
### Escolhendo o Torque como o f(x)

rpm_torque = (1400,0.163),(1800,0.192),(2200,0.203),(2600,0.199),(3000,0.190),(3400,0.180)
_x, _y = zip(*rpm_torque)


fig = go.Figure(data=go.Scatter(x=_x, y=_y, mode='markers',marker_color='red'))
fig.update_traces(mode='lines+markers', marker_line_width=2, marker_size=10)
fig.update_layout(title='Torque do Motor/RPM',yaxis_zeroline=False);fig.show()

In [7]:
from sympy import symbols, simplify, nsimplify, lambdify
from mpmath import fsum, fprod

def lagrange_eq(data):
  _x, _y = zip(*data)
  if len(_x) != len(_y): raise ValueError
  x,y = symbols('x'),0
  
  for i in range(len(_x)):
    y += fprod([((x - _x[j]) / (_x[i] - _x[j])) for j in range(len(_y)) if i!=j]) * _y[i]
    
  return y

rpm_torque = (1400,0.163),(1800,0.192),(2200,0.203),(2600,0.199),(3000,0.190),(3400,0.180)

interp_tq = lagrange_eq(rpm_torque)
display("Montando a Equação"); display(interp_tq); display("Simplificando"); display(nsimplify(interp_tq))
display(nsimplify(simplify(interp_tq))); display("Resultando em:"); display(simplify(interp_tq))

res_lagrange = lambdify('x', interp_tq)

res_lagrange(1600)

'Montando a Equação'

0.163*(17/10 - x/2000)*(15/8 - x/1600)*(13/6 - x/1200)*(11/4 - x/800)*(4.5 - 0.0025*x) + 0.192*(17/8 - x/1600)*(5/2 - x/1200)*(13/4 - x/800)*(11/2 - x/400)*(0.0025*x - 3.5) + 0.203*(17/6 - x/1200)*(15/4 - x/800)*(13/2 - x/400)*(0.00125*x - 1.75)*(x/400 - 9/2) + 0.199*(17/4 - x/800)*(15/2 - x/400)*(0.000833333333333333*x - 1.16666666666667)*(x/800 - 9/4)*(x/400 - 11/2) + 0.19*(17/2 - x/400)*(0.000625*x - 0.875)*(x/1200 - 3/2)*(x/800 - 11/4)*(x/400 - 13/2) + 0.18*(0.0005*x - 0.7)*(x/1600 - 9/8)*(x/1200 - 11/6)*(x/800 - 13/4)*(x/400 - 15/2)

'Simplificando'

163*(17/10 - x/2000)*(15/8 - x/1600)*(13/6 - x/1200)*(11/4 - x/800)*(9/2 - x/400)/1000 + 24*(17/8 - x/1600)*(5/2 - x/1200)*(13/4 - x/800)*(11/2 - x/400)*(x/400 - 7/2)/125 + 203*(17/6 - x/1200)*(15/4 - x/800)*(13/2 - x/400)*(x/800 - 7/4)*(x/400 - 9/2)/1000 + 199*(17/4 - x/800)*(15/2 - x/400)*(x/1200 - 7/6)*(x/800 - 9/4)*(x/400 - 11/2)/1000 + 19*(17/2 - x/400)*(x/1600 - 7/8)*(x/1200 - 3/2)*(x/800 - 11/4)*(x/400 - 13/2)/100 + 9*(x/2000 - 7/10)*(x/1600 - 9/8)*(x/1200 - 11/6)*(x/800 - 13/4)*(x/400 - 15/2)/50

-105794270833337*x**5/10000000000000000000000000000000 + 127766927083341*x**4/1000000000000000000000000000 - 36682128906251*x**3/62500000000000000000000 + 31027018229167*x**2/25000000000000000000 - 113650651041669*x/100000000000000000 + 124538085937509/250000000000000

'Resultando em:'

-1.05794270833337e-17*x**5 + 1.27766927083341e-13*x**4 - 5.86914062500016e-10*x**3 + 1.24108072916668e-6*x**2 - 0.00113650651041669*x + 0.498152343750036

0.17930859374999766

In [9]:

def newton(data):
  _x, __y = zip(*data)
  if len(_x) != len(__y): raise ValueError
  x = symbols('x')  
  n = len(_x)
  _y = [[0 for _ in range(len(_x))] for _ in range(len(_x))]
  
  for i in range(n):
    _y[i][0] = __y[i]

  for i in range(1,n):
    for j in range(i,n):
      _y[j][i] = ((_y[j][i-1]-_y[j-1][i-1]) / (_x[j]- _x[j-i]))

  display(HTML(math('DD',np.array(_y))))

  a = 0
  for i in range(n):
    mul = _y[i][i];
    for j in range(1,i+1):
        mul *= (x - _x[j-1])
    a += mul
  
  return a

rpm_torque = (1400,0.163),(1800,0.192),(2200,0.203),(2600,0.199),(3000,0.190),(3400,0.180)

interp_tq = newton(rpm_torque)
display("Montando a Equação"); display(interp_tq); display("Simplificando"); display(nsimplify(interp_tq))
display(nsimplify(simplify(interp_tq))); display("Resultando em:"); display(simplify(interp_tq))

res_newton = lambdify('x',interp_tq)
res_newton(1600)

'Montando a Equação'

7.25e-5*x + (1.48111979166667e-14 - 1.05794270833334e-17*x)*(x - 3000)*(x - 2600)*(x - 2200)*(x - 1800) + (7.875e-5 - 5.625e-8*x)*(x - 1800) + (1.13932291666667e-14*x - 1.59505208333334e-11)*(x - 2600)*(x - 2200)*(x - 1800) + (7.81249999999994e-12*x - 1.09374999999999e-8)*(x - 2200)*(x - 1800) + 0.0615

'Simplificando'

29*x/400000 + (148111979166667/10000000000000000000000000000 - 52897135416667*x/5000000000000000000000000000000)*(x - 3000)*(x - 2600)*(x - 2200)*(x - 1800) + (63/800000 - 9*x/160000000)*(x - 1800) + (113932291666667*x/10000000000000000000000000000 - 79752604166667/5000000000000000000000000)*(x - 2600)*(x - 2200)*(x - 1800) + (390624999999997*x/50000000000000000000000000 - 109374999999999/10000000000000000000000)*(x - 2200)*(x - 1800) + 123/2000

-52897135416667*x**5/5000000000000000000000000000000 + 63883463541667*x**4/500000000000000000000000000 - 293457031250001*x**3/500000000000000000000000 + 124108072916667*x**2/100000000000000000000 - 113650651041667*x/100000000000000000 + 498152343750003/1000000000000000

'Resultando em:'

-1.05794270833334e-17*x**5 + 1.27766927083334e-13*x**4 - 5.86914062500002e-10*x**3 + 1.24108072916667e-6*x**2 - 0.00113650651041667*x + 0.498152343750003

0.17930859375

In [10]:
# novos valores intermediaros

rpm_torque = (1400,0.163),(1800,0.192),(2200,0.203),(2600,0.199),(3000,0.190),(3400,0.180)
new_torque = (1400,1600, 1700, 2100, 2500,3100)
lag_torque = (1600, 1900, 2300, 2800, 3200)
lagrange_torque = [(x, res_lagrange(x)) for x in lag_torque]
newton_torque = [(x, res_newton(x)) for x in new_torque]
all_torque = [(x,res_lagrange(x)) for x in range(1200, 3500)]

_x1, _y1 = zip(*rpm_torque)
_x2, _y2 = zip(*lagrange_torque)
_x3, _y3 = zip(*newton_torque)
_x4, _y4 = zip(*all_torque)

fig = go.Figure(data=go.Scatter(x=_x1, y=_y1, mode='markers',marker_color='red', name='sensores'))
fig.add_trace(go.Scatter(x=_x2, y=_y2, mode='markers',marker_color='blue',name='lagrange'))
fig.add_trace(go.Scatter(x=_x3, y=_y3, mode='markers',marker_color='pink',name='newton'))
fig.add_trace(go.Scatter(x=_x4, y=_y4, mode='lines',marker_color='green', name='lagrange (linha)'))

fig.update_traces(marker_line_width=2, marker_size=10)
fig.update_layout(title='Torque do Motor/RPM',yaxis_zeroline=False);fig.show()

In [11]:
#curvas
rpm_torque = (1400,0.163),(1800,0.192),(2200,0.203),(2600,0.199),(3000,0.190),(3400,0.180)


def least_squares(data):
  x,y = zip(*data)
  N = len(x)
  x_avg = sum(x)/N
  y_avg = sum(y)/N
  var_x, cov_xy = 0, 0
  
  for x,y in zip(x,y):
      temp = x - x_avg
      var_x += temp**2
      cov_xy += temp * (y - y_avg)
  slope = cov_xy / var_x
  y_interc = y_avg - slope*x_avg
  return (slope, y_interc)

sl, intr = least_squares(rpm_torque)

_x6, _= zip(*rpm_torque)
_x6 = np.array(_x6)

poly = sl * _x6 + intr

fig = go.Figure(data=go.Scatter(x=_x6, y=poly, mode='markers + lines',marker_color='red', name='Linha (menor quadrado)'))
fig.add_trace(go.Scatter(x=_x2, y=_y2, mode='markers',marker_color='blue',name='lagrange'))
fig.update_traces(marker_line_width=1, marker_size=10)
fig.update_layout(title='Integral Numérica (partições)',yaxis_zeroline=False);fig.show()


In [12]:
# novos valores intermediarios

rpm_torque = (1400,0.163),(1800,0.192),(2200,0.203),(2600,0.199),(3000,0.190),(3400,0.180)
new_torque = (1400,1600, 1700, 2100, 2500,3100)
lag_torque = (1600, 1900, 2300, 2800, 3200)
lagrange_torque = [(x, res_lagrange(x)) for x in lag_torque]
newton_torque = [(x, res_newton(x)) for x in new_torque]
all_torque = [(x,res_lagrange(x)) for x in range(1200, 3500)]
trapezoid = [ (1600, res_lagrange(1600)),(3400,res_lagrange(3400))]

_x1, _y1 = zip(*rpm_torque)
_x2, _y2 = zip(*lagrange_torque)
_x3, _y3 = zip(*newton_torque)
_x4, _y4 = zip(*all_torque)
_x5, _y5 = zip(*trapezoid)

fig = go.Figure(data=go.Scatter(x=_x1, y=_y1, mode='markers',marker_color='red', name='sensores'))
fig.add_trace(go.Scatter(x=_x2, y=_y2, mode='markers',marker_color='blue',name='lagrange'))
fig.add_trace(go.Scatter(x=_x3, y=_y3, mode='markers',marker_color='pink',name='newton'))
fig.add_trace(go.Scatter(x=_x4, y=_y4, mode='lines',marker_color='green', name='lagrange (linha)'))
fig.add_trace(go.Scatter(x=_x1, y=_y1, mode='lines',fill='tozeroy',marker_color='red', name='INTEGRAL'))


fig.update_traces(marker_line_width=2, marker_size=10)
fig.update_layout(title='IntegralTorque do Motor/RPM',yaxis_zeroline=False);fig.show()

In [13]:
# INTEGRAL
import numpy as np
import pandas as pd
from sympy import integrate

def trap(f,a,b,N=50):
  
  x = np.linspace(a,b,N+1)
  y = f(x)
  y_r, y_l = y[1:], y[:-1]
  dx = (b - a)/N
  T = (dx/2) * np.sum(y_r + y_l)
  return T

#trap(lambda x: x**2, 0,10)

res_itg = []

for i in range(1,50):
  res_itg.append(trap(res_lagrange, 1200, 3200,N=i))

_y6 = [x for x in range(50)]

display('função original')
display(simplify(interp_tq))
display('integral simbólica')
display(integrate(interp_tq))
ig = lambdify('x',integrate(interp_tq))

intg = ig(3200) - ig(1200)
print(f"(integral analítica) = {intg}")
print(f"integral numérica - 100 partições {trap(res_lagrange,1200,3200, N=100)}")
fig = go.Figure(data=go.Scatter(x=_y6, y=res_itg, mode='markers + lines',marker_color='red', name='sensores'))

fig.update_traces(marker_line_width=1, marker_size=10)
fig.update_layout(title='Integral Numérica (partições)',yaxis_zeroline=False);fig.show()

'função original'

-1.05794270833334e-17*x**5 + 1.27766927083334e-13*x**4 - 5.86914062500002e-10*x**3 + 1.24108072916667e-6*x**2 - 0.00113650651041667*x + 0.498152343750003

'integral simbólica'

-1.76323784722223e-18*x**6 + 2.55533854166667e-14*x**5 - 1.46728515625001e-10*x**4 + 4.13693576388891e-7*x**3 - 0.000568253255208336*x**2 + 0.498152343750003*x

(integral analítica) = 378.09201388883787
integral numérica - 100 partições 378.0885589062461


### Referências:
##### T. Stoeck, K. F. Abramek: APPLICATION OF THE POLYNOMIAL INTERPOLATION METHOD FOR DETERMINING PERFORMANCE CHARACTERISTICS OF A DIESEL ENGINE
##### Metrol. Meas. Syst., Vol. XXI (2014), No. 1, pp. 157–168.